In [3]:
from sage.algebras.flag_algebras import *
import itertools

# Create the theory
def test_identify(n, ftype_points, edges, C0, C1, C2):
    return colored_identify(2, [[0], [1], [2]], n, ftype_points, edges=edges, C0=C0, C1=C1, C2=C2)

def test_generate(n):
    return colored_generate(2, [[0], [1], [2]], n)

TT = CombinatorialTheory("123graphs", test_generate, test_identify, edges=2, C0=1, C1=1, C2=1)

feasibles = []
for flag in TT.generate_flags(4):
    # Real edges
    edge_list = [sorted(x + [4]) for x in flag.blocks()['edges']]
    # Add rainbow edges
    for v1 in flag.blocks()['C0']:
        for v2 in flag.blocks()['C1']:
            for v3 in flag.blocks()['C2']:
                l = sorted([v1[0], v2[0], v3[0]])
                if l not in edge_list:
                    edge_list.append(l)
    # Look for C5-
    feasible = True
    for p in list(itertools.permutations([0, 1, 2, 3, 4])):
        if sorted([p[0], p[1], p[2]]) in edge_list and sorted([p[1], p[2], p[3]]) in edge_list and sorted([p[2], p[3], p[4]]) in edge_list and sorted([p[3], p[4], p[0]]) in edge_list:
            feasible = False
            break
    # Account for this flag
    if feasible:
        feasibles.append(flag)

exclude = [flag for flag in TT.generate_flags(4) if flag not in feasibles]
TT.exclude(exclude)
print(len(feasibles))

279


In [4]:
# Assumptions
edge_00 = TT(2, edges=[[0, 1]], C0=[[0], [1]], C1=[], C2=[])
edge_11 = TT(2, edges=[[0, 1]], C0=[], C1=[[0], [1]], C2=[])
edge_22 = TT(2, edges=[[0, 1]], C0=[], C1=[], C2=[[0], [1]])
edge_01 = TT(2, edges=[[0, 1]], C0=[[0]], C1=[[1]], C2=[])
edge_12 = TT(2, edges=[[0, 1]], C0=[], C1=[[0]], C2=[[1]])
edge_02 = TT(2, edges=[[0, 1]], C0=[[0]], C1=[], C2=[[1]])
point0 = TT(1, edges = [], C0 = [[0]], C1 = [], C2 = [])
point1 = TT(1, edges = [], C0 = [], C1 = [[0]], C2 = [])
point2 = TT(1, edges = [], C0 = [], C1 = [], C2 = [[0]])
positives = [edge_12 - edge_01, edge_12 - edge_02, point0 - 1/3, point1 - 1/3, point2 - 1/3]

# # forall i, d(v_i) <= 1/4
positives += [1/36 - edge_00, 1/36 - edge_11, 1/36 -  edge_22]

# # d(v) >= 1 / 4
positives += [edge_00 + edge_11 + edge_22 + edge_01 + edge_12 + edge_02 - 1/4]

# Third
positives += [edge_12 - 1/100]

In [5]:
# Missing edges
M = 1 + TT(2, edges=[], C0=[], C1=[[0]], C2=[[1]]) - 1

# Bad edges
B = 1
B += TT(2, edges=[[0, 1]], C0=[[0]], C1=[[1]], C2=[])
B += TT(2, edges=[[0, 1]], C0=[[0]], C1=[], C2=[[1]])
B += TT(2, edges=[[0, 1]], C0=[], C1=[[0], [1]], C2=[])
B += TT(2, edges=[[0, 1]], C0=[], C1=[], C2=[[0], [1]])
B -= 1

In [6]:
# Optimize
# const = TT.blowup_construction(5, 3, edges=[[1, 2]], C0=[[0]], C1=[[1]], C2=[[2]])
# x = TT.optimize(B - M, 5, maximize=True, positives = positives, exact=True, construction=const)
# print(x)

x = TT.optimize(B - M, 5, maximize=True, positives = positives)
print(x)

Base flags generated, their number is 1695
The relevant ftypes are constructed, their number is 59
Block sizes before symmetric/asymmetric change is applied: [42, 42, 42, 24, 24, 24, 24, 15, 24, 24, 24, 24, 24, 24, 20, 20, 18, 20, 9, 18, 9, 20, 18, 9, 24, 20, 18, 20, 18, 18, 24, 24, 18, 18, 16, 16, 5, 16, 5, 16, 18, 24, 18, 5, 16, 16, 18, 18, 24, 24, 16, 16, 16, 3, 16, 24, 16, 16, 24]


Done with mult table for Ftype on 3 points with edges=[[0, 1], [0, 2], [1, 2]], C0=[], C1=[], C2=[[0], [1], [2]]: : 59it [00:00, 73.92it/s]


Tables finished


Done with positivity constraint 9: 100%|████████| 10/10 [00:01<00:00,  6.81it/s]


Constraints finished
Running sdp without construction. Used block sizes are [42, 42, 42, 12, 12, 18, 6, 18, 6, 18, 6, 15, 18, 6, 12, 12, 18, 6, 18, 6, 12, 12, 18, 6, 20, 20, 14, 4, 20, 9, 14, 4, 9, 20, 14, 4, 9, 18, 6, 20, 14, 4, 20, 14, 4, 14, 4, 18, 6, 18, 6, 14, 4, 14, 4, 16, 16, 5, 16, 5, 16, 14, 4, 18, 6, 14, 4, 5, 16, 16, 14, 4, 14, 4, 18, 6, 12, 12, 12, 4, 12, 4, 12, 4, 3, 12, 4, 12, 12, 12, 4, 12, 4, 12, 12, -1695, -1231]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 6.48e-01 Pobj: -5.4293765e+01 Ad: 3.71e-01 Dobj:  6.0352468e+01 
Iter:  2 Ap: 1.00e+00 Pobj: -1.3392549e+02 Ad: 6.42e-01 Dobj:  4.8088202e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -1.4273210e+02 Ad: 8.99e-01 Dobj:  8.3866682e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.4806569e+02 Ad: 9.36e-01 Dobj: -3.4456821e-02 
Iter:  5 Ap: 1.00e+00 Pobj: -1.3853189e+02 Ad: 8.38e-01 Dobj: -7.4345592e-02 
Iter:  6 Ap: 3.85e-01 Pobj: -1.2977321e+02 Ad: 6.30e-01 Dobj: -4.7854117e-02 
Iter:  7 